In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data/student_performance_with_PS_and_labels.csv')

In [3]:
df.head()

,last_sem_spi,internal_assessment_avg,attendance_percentage,total_backlogs,pyq_solving_frequency,study_hours_weekly,sleep_category,gaming_hours_weekly,assignment_delay_count,department,travel_time_category,extra_curricular_level,sleep_score,travel_score,extra_score,ps,performance_category
0,4.37,73.8,43.9,3,0,46.9,<6,6.9,3,EE,30-60,Low,0.3,0.7,0.3,38.25,At Risk
1,9.56,41.1,27.6,1,0,9.0,6-8,17.8,0,ME,>60,High,1.0,0.4,1.0,43.60,Average
2,7.59,50.8,30.1,1,1,16.8,>8,15.2,2,EE,<30,Low,0.8,1.0,0.3,37.26,At Risk
3,6.39,69.8,34.5,0,3,17.3,6-8,18.3,1,EE,30-60,High,1.0,0.7,1.0,55.70,Average
4,2.40,58.9,36.3,2,2,34.4,<6,7.9,2,EC,<30,Medium,0.3,1.0,0.7,28.89,At Risk


In [4]:
model_df = df.drop(columns=['ps'])
model_df.head()

,last_sem_spi,internal_assessment_avg,attendance_percentage,total_backlogs,pyq_solving_frequency,study_hours_weekly,sleep_category,gaming_hours_weekly,assignment_delay_count,department,travel_time_category,extra_curricular_level,sleep_score,travel_score,extra_score,performance_category
0,4.37,73.8,43.9,3,0,46.9,<6,6.9,3,EE,30-60,Low,0.3,0.7,0.3,At Risk
1,9.56,41.1,27.6,1,0,9.0,6-8,17.8,0,ME,>60,High,1.0,0.4,1.0,Average
2,7.59,50.8,30.1,1,1,16.8,>8,15.2,2,EE,<30,Low,0.8,1.0,0.3,At Risk
3,6.39,69.8,34.5,0,3,17.3,6-8,18.3,1,EE,30-60,High,1.0,0.7,1.0,Average
4,2.40,58.9,36.3,2,2,34.4,<6,7.9,2,EC,<30,Medium,0.3,1.0,0.7,At Risk


In [5]:
categorical_cols = [
    "sleep_category",
    "department",
    "travel_time_category",
    "extra_curricular_level",
    "performance_category"
]

In [6]:
encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    model_df[col] = le.fit_transform(model_df[col])
    encoders[col] = le

In [7]:
X = model_df.drop(columns=["performance_category"])
y = model_df["performance_category"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Random Forest Model

In [8]:
rf_model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight="balanced"
)
rf_model.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [9]:
rf_model.score(X_test, y_test)

0.8955

In [10]:
feature_importance = pd.DataFrame({
    "feature": X.columns,
    "importance": rf_model.feature_importances_
}).sort_values(by="importance", ascending=False)

In [11]:
feature_importance.head()

,feature,importance
0,last_sem_spi,0.301224
1,internal_assessment_avg,0.215320
2,attendance_percentage,0.138791
5,study_hours_weekly,0.084314
4,pyq_solving_frequency,0.067024


In [12]:
models = {
    "rf": RandomForestClassifier(
            n_estimators=400,
            max_depth=10,
            random_state=42,
            class_weight="balanced"
        ),
    "svm": SVC(kernel='rbf'),
    "perceptron": Perceptron(
        penalty='elasticnet',
        l1_ratio=0.85,
        alpha=0.05,
        max_iter=2500
    ),
    "xgb": XGBClassifier(
        n_estimators=500,
        max_depth=5,
        random_state=42,
        n_jobs=-1
    ),
    "lgbm": LGBMClassifier(
        boosting_type='rf',
        bagging_freq=1,
        bagging_fraction=0.8,
        max_depth=10,
        learning_rate=0.1,
        n_estimators=400,  
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    )
}

In [13]:
accuracies = []
for name, algo in models.items():
    algo.fit(X_train, y_train)
    accuracy = algo.score(X_test, y_test) * 100
    print(f"{name.upper()} | Accuracy: {accuracy}")
    accuracies.append(accuracy)

RF | Accuracy: 90.2
SVM | Accuracy: 91.475
PERCEPTRON | Accuracy: 8.1
XGB | Accuracy: 96.125
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003634 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1296
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 15
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[Light

In [14]:
summary_df = pd.DataFrame({
    "name": models.keys(),
    "accuracies": accuracies
})
summary_df

,name,accuracies
0,rf,90.200
1,svm,91.475
2,perceptron,8.100
3,xgb,96.125
4,lgbm,76.950
